# Heart Failure Prediction

**Reference**
- [Heart Fail:Analysis and Quick-prediction (NAYAN SAKHIYA)](https://www.kaggle.com/code/nayansakhiya/heart-fail-analysis-and-quick-prediction)

In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


## Data Preprocessing

In [12]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

df = pd.read_csv('./heart_failure_clinical_records_dataset.csv')
df = df.drop(df.loc[df['creatinine_phosphokinase']>3000].index, axis=0)
# df = pd.get_dummies(df)

float_columns = [x for x in df.columns if x not in ['anaemia', 'diabetes', 'high_blood_pressure', 'sex', 'smoking', 'DEATH_EVENT']]
sc = StandardScaler()
df2 = df.copy()
df[float_columns] = sc.fit_transform(df[float_columns])

#df.to_csv('Heart_Failure_scaled.csv', encoding='utf-8-sig')

X = df.drop(['DEATH_EVENT'], axis=1)
T = df['DEATH_EVENT']
X.head(10)

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time
0,1.201844,0,0.223304,0,-1.528286,1,0.020103,0.519997,-1.493845,1,0,-1.643285
2,0.352251,0,-0.586694,0,-1.528286,0,-1.026546,-0.083297,-1.720915,1,1,-1.604649
3,-0.922138,1,-0.651717,0,-1.528286,0,-0.538788,0.519997,0.095650,1,0,-1.604649
4,0.352251,1,-0.560685,1,-1.528286,0,0.650125,1.324389,-4.672833,0,0,-1.591771
5,2.476233,1,-0.770615,0,0.161451,1,-0.599757,0.721095,-1.039703,1,1,-1.591771
6,1.201844,1,-0.400914,0,-1.950720,0,-1.382204,-0.183846,0.095650,1,0,-1.566014
7,-0.072545,1,-0.272727,1,1.851187,0,1.940653,-0.284395,-1.266774,1,1,-1.566014
8,0.352251,0,-0.566258,0,2.273621,0,0.003418,0.117801,0.322720,0,0,-1.566014
9,1.626640,1,-0.629423,0,-0.260983,1,1.269985,8.061176,-0.812633,1,1,-1.566014
10,1.201844,1,-0.707450,0,-0.007523,1,1.066752,2.631527,-1.266774,1,1,-1.566014


## Train Test Split

In [13]:
from sklearn.model_selection import train_test_split

Features = ['age', 'ejection_fraction', 'serum_creatinine']
x_train, x_test, y_train, y_test = train_test_split(X, T, test_size=0.2, random_state=40)

## Modeling

### Logistic Regression

In [27]:
from sklearn.linear_model import LogisticRegression

### Support Vector

In [28]:
from sklearn.svm import SVC

### K Neighbors Classifier

In [29]:
from sklearn.neighbors import KNeighborsClassifier

### Decision Tree Classifier

In [30]:
from sklearn.tree import DecisionTreeClassifier

### Random Forest Classifier

In [31]:
from sklearn.ensemble import RandomForestClassifier

### Gradient Boosting Classifier

In [32]:
from sklearn.ensemble import GradientBoostingClassifier

### XGBRF Classifier

In [37]:
import xgboost as xgb

### LGBM Classifier


[**Reference...**](https://nurilee.com/2020/04/03/lightgbm-definition-parameter-tuning/)
- **Gradient Boosting Framework**
- ML Algorithm based on **Tree Model**
    - Light GBM: **Tree가 수직적으로 확장**
        - Leaf-wise 방식
        <img src="./Leaf_wise_tree_growth.png" width="500px" height="400px" title="Leaf_wise"></img>
    - Others: **Tree가 수평적으로 확장**
        - Level-wise 방식
        <img src="./Level_wise_tree_growth.png" width="400px" height="250px" title="Level_wise"></img>

- **Light GBM이 인기를 얻게된 이유**
    - 속도가 빠름
    - 큰 사이즈의 데이터를 다룰 수 있으며 적은 메모리 공간을 차지
    - 정확도가 높음
    - GPU gkrtmqdmf wldnjs
- **Light GBM 사용처**
    - 작은 데이터셋에는 부적합
    - Overfitting에 민감
    - row size에 대한 제한은 없으나 10,000개 이상에 권장
- **Light GBM 구현**
    - Parameter tuning이 복잡 (100개 이상)

**Parameters**  
- `max_depth`: 
    - Tree의 최대 깊이. 과적합을 해결할 때 사용
- `min_data_in_leaf`: 
    - Leaf가 가지고 있는 최소한의 레코드 수
    - 과적합을 해결할 때 사용
    - *deafult value 20
- `feature_fraction`:
    - Boosting이 Random Forest일 경우 사용
    - **0.8 feature_fraction**
        - Light GBMdl Tree를 만들 때 매번 각각의 iteration 반복에서 파라미터 중에 80%를 랜덤하게 선택
- `bagging_fraction`:
    - 매번 iteration을 돌 때 사용되는 데이터의 일부를 선택하는데 트레이닝 속도를 높이고 과적합을 방지할 때 주로 사용
- `early_stopping_round`:
    - 분석 속도를 높이는데 도움을 줌
    - 모델은 만약 어떤 validation 데이터 중 하나의 지표가 지난 early_stopping_round 라운드에서 향상되지 않았다면 학습을 중단함
        - 이는 지나친 iteration을 줄이는데 도움을 줌
- `lambda`:
    - lambda값은 regularization 정규화를 함
    - 0~1 사이
- `min_gain_to_split`:
    - 분기하기 위해 필요한 최소한의 gain을 의미
    - Tree의 유용한 분기수를 컨트롤하는데 사용
- `max_cat_group`:
    - 카테고리 수가 클 때, 과적합을 방지하는 분기포인트를 찾음
    - Light GBM 알고리즘이 카테고리 그룹을 max_cat_group 그룹으로 합치고 그룹 경계선에서 분기 포인트를 찾음
    - *default value 64
    
**Core Parameters**
- `Task`:
    - 데이터에 대해서 수행하고자 하는 임무를 구체화
    - training or prediction
- `application`:
    - 가장 중요한 파라미터로, 모델의 어플리케이션을 정함
    - regression or classification
    - dafault value $\text{regression}$
        - regression: 회귀분석
        - binary: 이진 분류
        - multiclass: 다중 분류
- `boosting`:
    - 실행하고자 하는 알고리즘 타입을 정의

In [34]:
import lightgbm

ModuleNotFoundError: No module named 'lightgbm'

### Cat Boost Classifier

In [35]:
from catboost import CatBoostClassifier

ModuleNotFoundError: No module named 'catboost'